__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-22 09:43:36 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-22 09:43:36 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-22 09:43:39 | AdsListParser: downloader init
[INFO    ] 2022-09-22 09:43:39 | CianParser: downloader init
[INFO    ] 2022-09-22 09:43:39 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-22 09:43:54 | AdsListParser: read page 1
[INFO    ] 2022-09-22 09:44:00 | AdsListParser: read page 2
[INFO    ] 2022-09-22 09:44:08 | AdsListParser: read page 3
[INFO    ] 2022-09-22 09:44:18 | AdsListParser: read page 4
[INFO    ] 2022-09-22 09:44:31 | AdsListParser: read page 5
[INFO    ] 2022-09-22 09:44:42 | AdsListParser: read page 6
[INFO    ] 2022-09-22 09:44:58 | AdsListParser: read page 7
[INFO    ] 2022-09-22 09:45:10 | AdsListParser: read page 8
[INFO    ] 2022-09-22 09:45:21 | AdsListParser: read page 9
[INFO    ] 2022-09-22 09:45:30 | AdsListParser: read page 10
[INFO    ] 2022-09-22 09:45:44 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
225,"Апартаменты-студия, 25 м², 5/5 этаж",,,5 700 000 ₽,228 000 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Стрелецкая бухта, улица Рыбацкий Причал, 6к2]","[вчера, вчера, 17:55]",https://sevastopol.cian.ru/sale/flat/269566437/,"Продам море удачи или дачу у моря!\nВ продаже апартаменты на берегу Стрелецкой бухты города Севастополя.\nАпартаменты - студия площадью 25 м2 с кухонной зоной\nСанузел совмещенный.\nВыполнен косметический ремонт. Студия укомплектована всей необходимой мебелью и техникой. Для обеспечения горячей водой установлен бойлер, контроль температуры помещения - кондиционер.\nДанные апартаменты расположены на закрытой территории Яхт-клуба с круглосуточной охраной на первой линии от моря. Отлично подходят для отдыха и проживания в чудесном районе на берегу Черного моря,\nАпартамены подходят для круглогодичной сдачи в аренду.\nОтличный вариант для владельцев маломерных судов и любителей рыбалки!\nВ шаговой доступности конечная остановка городского маршрутного транспорта (автобус 400), продуктовые магазины.\nБлижайшие пляжи: оборудованные - Песочный и Солнечный, дикий пляж -Красные камни.\nЛучшие места для проведения отдыха на берегу в данной локации: парк Победы, Сквер севастопольских курсантов, парк и. А.Ахматовой!\nБЕЗ КОМИССИИ ДЛЯ ПОКУПАТЕЛЯ!\nЗВОНИТЕ СРОЧНО! Оперативно покажем красоту и комфорт данного уголка Севастополя!\n\nЛот 204521",9,2022-09-22 09:53:39.964462
241,"2-комн. кв., 66,2 м², 5/6 этаж",,,12 900 000 ₽,194 864 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Стрелецкая бухта, улица Тульская, 3]","[30 минут назад, сегодня, 09:07]",https://sevastopol.cian.ru/sale/flat/277421171/,"Лучшее предложение по цене! Акция действует до 05.10.2022 г.!!!\n\nИнфраструктура: \n\nУдобное месторасположение, вблизи исторического центра города и административного центра. В шаговой доступности: хорошая школа-гимназия 24 со своим плавательным бассейном, детский сад, Медицинский колледж, школа-лицей, поликлиника, рынок, магазины, супермаркет БУМ, ТЦ Муссон, Лаванда, спортивный комплекс, кинотеатр и т.д. В пешей доступности расположены музей-заповедник Херсонес Таврический, а также лучшие городские пляжи Песочный и Солнечный, новый эстетичный парк им. Анны Ахматовой. Комфортная транспортная развязка, легко можно добраться в любую часть города или быстро выехать из него. \n\nХарактеристика: \n\nДом 2006 г., 6-этажный, из инкерманского блока шириной 60 см, застройщик строительная компания ""Агат-А"". \n\nДвор просторный, много мест для парковки не только Вам, но и Вашим гостям. Чистый благоустроенный, в ближайшей перспективе закрытый, размещены видеокамеры по периметру для Вашего спокойствия и безопасности (все взносы произведены). Установлен видеодомофон. \n\nЧистый уютный подъезд. Всего 3 квартиры на этаже. 5-й этаж идеален, не течет крыша, никто не стучит в окна. \n\n2- комнатная квартира, расположение комнат угловая бабочка обзор на две стороны дома. \n\nКачественный евроремонт произведен в 2015 г.: на стенах венецианская штукатурка, натяжной потолок. На полу ламинат, в коридоре и санузле кафель. Квартира в безупречном состоянии, полностью готова для Вас. \n\nОстается полностью оборудованная кухня: встроенная мебель, духовая плита, вытяжка, Качественная мебель в комнатах, 2 кондиционера: в спальне и на кухне. Произведена перепланировка: спальня и кухня совмещены с балконами. Есть гардеробная, что очень экономит пространство и облегчает жизнь. В подвальном помещении своя ячейка 17 м2. \n\nКоммуникации: двухконтурный газовый котел Pantera с терморегулятором (по всей квартире установлены радиаторы). \n\nЭта квартира - идеальный баланс между функциональностью, красотой и комфортом. Интерьер квартиры в сдержанной гамме, которая поддерживает стиль открытого пространства и много воздуха. \n\nДокументы готовы для сделки, любая форма расчета. \n\nС удовольствием проведем показ! Всегда на связи!\n\nЛот 731",9,2022-09-22 09:53:39.964462
1

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1509
1473


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [10]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [11]:
# df

In [12]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [13]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [14]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])